### Семинар 6. Контейнеры и итераторы.

<br />

##### begin && end

`begin()`, `end()` - методы у контейнеров, возвращающие итераторы:
* `begin()` на первый элемент в контейнере
* `end()` на следующий за последним элементом

Как, зная `begin()` и `end()`, проверить, что контейнер пуст?

```c++
std::vector<int> v = {10, 20, 30, 40};
```

![](vector_internals.png)

```c++
auto it = v.begin();
std::cout << *it; // 10
        
++it;
std::cout << *it; // 20
        
++it;
std::cout << *it; // 30

++it;
std::cout << *it; // 40

++it; // it == v.end()
std::cout << *it; // UB - you should never dereference end()!
```

<br />

##### Итераторы и ассоциативные контейнеры

Обратить внимание на сложности итерирования по ассоциативным контейнерам

![](unordered_internals.jpg)

<br />

##### range for

https://en.cppreference.com/w/cpp/language/range-for

```c++
std::vector<int> v = {10, 20, 30, 40};
for (int x : v)
    std::cout << x; // во что разворачивается range-for? (упрощённо)
```

во что разворачивается range-for? (упрощённо)

```c++
for (auto x : v) { ... }
```

```c++
{
    auto&& __range = v;
    auto __begin = __range.begin();
    auto __end = __range.end();
    for (; __begin != __end; ++__begin) {
        auto x = *__begin;
        ...
    }
}
```

Смысл в том, что как только для пользовательского контейнера определены итераторы и методы `begin()`, `end()`, `cbegin()`, `cend()`, для него "из коробки" начинает работать range-for.

<br />

##### Инвалидация итераторов

Итераторы могут быть инвалидированы, если контейнер меняется.

Рассмотрим пример `std::vector` в предположении, что итератор - указатель на элемент в `std::vector`.

(нарисовать происходящее на доске)

```c++
std::vector<int> v = {10, 20, 30, 40};

auto v_end = v.end();
auto it = v.begin();

v.push_back(50);
// at this point:
// |it|    - invalidated
// |v_end| - invalidated

std::cout << *it; // oooops, ub
if (v.begin() == v_end)  // ooooops, ub
    ...;
```

https://en.cppreference.com/w/cpp/container/vector/push_back

```c++
std::set<int> s = {20, 30, 40, 50};
        
auto it = s.begin();
std::cout << *it;  // 20

s.insert(10);

std::cout << *it;  // ok, 20
```

Почему так? Потому что документация:

https://en.cppreference.com/w/cpp/container/set/insert

*У каждого контейнера у каждого метода прописан контракт на валидность итераторов (когда и какие итераторы инвалидируются).*

*Читайте документацию внимательно!*

<br />

##### Правильное удаление элементов из map/set/vector/... по условию

Как неправильно удалять элементы из `std::set`:

```c++
std::set<int> s = {1, 2, 3, 4, 5};

auto it = s.begin();
auto e = s.end();
for(; it != e; ++it)
    if((*it) % 2 == 1)
        s.erase(it);
```

В каком месте баг?

Правильное удаление:

```c++
std::set<int> s = {1, 2, 3, 4, 5};

auto it = s.begin();
auto e = s.end();
for(; it != e;)
{
    if((*it) % 2 == 1)
        it = s.erase(it);
    else
        s.erase(it);
}
```

Пример того, почему нельзя верить кому попало на stackoverflow:
https://stackoverflow.com/questions/800955/remove-if-equivalent-for-stdmap

<br />

##### Операции над итераторами доступа

```c++
std::vector<int> v = {10, 20, 30, 40, 50};
        
auto it = v.begin();

// некоторые итераторы позволяют брать следующий элемент:
auto jt_1 = it + 1;
auto jt_2 = std::next(it);
        
// некоторые итераторы позволяют брать предыдущий элемент:
auto jt_3 = it - 1;         // ?
auto jt_4 = std::prev(it);  // ?
        
// некоторые итераторы позволяют прыгать на n элементов вперёд:
auto jt_5 = it + 4;
auto jt_6 = std::advance(it, 4);

// некоторые итераторы позволяют прыгать на n элементов назад:
auto jt_7 = it - 4;                // ?
auto jt_8 = std::advance(it, -4);  // ?

// некоторые итераторы позволяют считать расстояние между ними:
std::cout << std::distance(it, jt_5);  // 4
```

Стандартные операции над итераторами доступа (access iterators):
* `std::next`
* `std::prev`
* `std::advance`
* `std::distance`

<br />

##### Типы итераторов

Как вы помните, у access iterators у `std::forward_list` нельзя делать `--it`, они только для хождения вперёд и только по одному шагу. А у `std::vector` можно вперёд-назад и на любое число шагов за раз. По этому принципу классифицируются итераторы доступа:
* [Forward Iterator](https://en.cppreference.com/w/cpp/named_req/ForwardIterator)
* [Bidirectional Iterator](https://en.cppreference.com/w/cpp/named_req/BidirectionalIterator)
* [Random Access Iterator](https://en.cppreference.com/w/cpp/named_req/RandomAccessIterator)

Классификация имеет важное значение для алгоритмов. Например, алгоритм сортировки работает только с Random Access Iterator:

```c++
std::vector<int> v = {20, 30, 10};
std::sort(v.begin(), v.end());  // ok
        
std::list<int> l = {20, 30, 10};
std::sort(l.begin(), l.end());  // compile-time error
```

И это отражено в требованиях к алгоритму:
    
https://en.cppreference.com/w/cpp/algorithm/sort

Поэтому для `std::list` реализовали свой `sort`:

https://en.cppreference.com/w/cpp/container/list/sort

```c++
std::list<int> l = {20, 30, 10};
l.sort();
```

<br />

Прочие типы итераторов:
* [Input Iterator](https://en.cppreference.com/w/cpp/named_req/InputIterator)
* [Output Iterator](https://en.cppreference.com/w/cpp/named_req/OutputIterator)
* [Reverse Iterator](https://en.cppreference.com/w/cpp/iterator/reverse_iterator)

Мощь и безобразие итераторов в двух примерах:

```c++
std::istringstream str("0.1 0.2 0.3 0.4");
const double sum = std::accumulate(std::istream_iterator<double>(str),
                                   std::istream_iterator<double>(),
                                   0.);


std::vector<int> v = {1, 2, 3, 4, 5};
std::copy(v.begin(),
          v.end(),
          std::ostream_iterator<int>(std::cout, " "));
```

<br />

##### reverse_iterator

![](reverse_iterator.jpg)

```c++
std::vector<int> v = {10, 20, 30, 40};

// обход в прямом направлении в стиле до С++11
std::copy(v.begin(),
          v.end(),
          std::ostream_iterator<int>(std::cout, " ")); // 10 20 30 40
            
// обход в обратном направлении:
std::copy(v.rbegin(),
          v.rend(),
          std::ostream_iterator<int>(std::cout, " ")); // 40 30 20 10
            
            
// сортировка по возрастанию:
std::sort(v.begin(), v.end());
        
// сортировка по убыванию:
std::sort(v.rbegin(), v.rend());
```

Конвертация iterator <-> reverse_iterator:

Обратите внимание на перескакивание итератора на предыдущий элемент при конвертации.

```c++
std::vector<int> v = {10, 20, 30, 40};
        
auto it = v.begin() + 2; // 30

auto rit = std::make_reverse_iterator(it); // 20 !!!!

auto it2 = rit.base();  // 30 !!!!
```

<br />

Задача: найти последнее число 5 в последовательности, предшествующее первому 10

Вариант решения:

```c++
template<typename It>
It function(It begin, It end)
{
    auto it = std::find(begin, end, 10);
    
    if (it == end)
        return end;  // no 10
    
    auto rit = std::find(std::make_reverse_iterator(it),
                         std::make_reverse_iterator(begin),
                         5);

    if (rit == std::make_reverse_iterator(begin))
        return end;  // no 5 before 10
    
    return std::next(rit.base());    
}

std::list<int> l = {1, 2, 3, 5, 5, 10};
auto it = function(l.begin(), l.end());
auto rit = function(l.rbegin(), l.rend());
```